In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

# set_log_level("ERROR")

In [1]:
!pip install git+https://github.com/ourownstory/neural_prophet.git --upgrade

  Cloning https://github.com/ourownstory/neural_prophet.git to /private/var/folders/t6/zc4n8b2j6rdcj9p7xb7gxm0h0000gn/T/pip-req-build-7pn8sdsn
  Running command git clone -q https://github.com/ourownstory/neural_prophet.git /private/var/folders/t6/zc4n8b2j6rdcj9p7xb7gxm0h0000gn/T/pip-req-build-7pn8sdsn
  Resolved https://github.com/ourownstory/neural_prophet.git to commit 02cbe0461453ef5d41d50808747609c892362bac
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85.2 MB 2.7 MB/s eta 0:00:011     |██████████████████████████▏     | 69.7 MB 17.1 MB/s eta 0:00:01
     |████████████████████████████████| 137.6 MB 59 kB/s s eta 0:00:01    |███████▋                        | 32.8 MB 22.0 MB/s eta 0:00:05     |███████████████████████████████▌| 135.5 MB 17.0 MB/s eta 0:00:01
  Created wheel for neuralprophet: filename=neura

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')
df

,ds,y
3692,2012-05-01,211
3691,2012-05-02,201
3690,2012-05-03,204
3689,2012-05-04,210
3688,2012-05-05,180
...,...,...
4,2022-06-07,276
3,2022-06-08,278
2,2022-06-09,284
1,2022-06-10,253


In [5]:
len(df)

3693

In [22]:
m = NeuralProphet(
  # yearly_seasonality=False,
  # weekly_seasonality=True,
  # daily_seasonality=False,
  n_lags=4*7+1,
  n_forecasts=8,
  changepoints_range=0.95,
  n_changepoints=50,
  num_hidden_layers=4,
  d_hidden=36,
  learning_rate=0.005,
).add_country_holidays("CA")

metrics = m.fit(df[df.ds>'2020-01-01'], 
                freq='D', 
                # progress='plot'
                )
print(metrics.tail(1))

with open('../models/daily_visits_forecast_model.pkl', "wb") as f:
    pickle.dump(m, f)


WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/utils.py:251: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  additive_events_dims = additive_events_dims.append(

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neur

     SmoothL1Loss       MAE       RMSE  RegLoss
203      0.002154  9.896804  12.998765      0.0


In [27]:
future = m.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False

forecast = m.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[0][0]
forecast_length = len(forecast.values.tolist()[0][1:])

forecast_output = pd.DataFrame(columns=['ds','visits','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='D')
forecast_output['visits'] = forecast.values.tolist()[0][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('../forecasts/daily_visits.csv', index=False)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:2311: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(future_df)

WARNING - (py.warnings._showwarnmsg) - /opt/python/3.10.4/lib/python3.10/site-packages/neuralprophet/forecaster.py:1649: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



In [28]:
forecast_output

,ds,visits,timestamp
0,2022-06-12,209.895889,2022-06-13 02:28:51
1,2022-06-13,289.959778,2022-06-13 02:28:51
2,2022-06-14,259.964844,2022-06-13 02:28:51
3,2022-06-15,253.703323,2022-06-13 02:28:51
4,2022-06-16,244.465210,2022-06-13 02:28:51
5,2022-06-17,246.489777,2022-06-13 02:28:51
6,2022-06-18,202.001556,2022-06-13 02:28:51
7,2022-06-19,200.629593,2022-06-13 02:28:51


In [7]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import pickle

# set_log_level("ERROR")

loaded_model = pickle.load(open('../models/daily_visits_forecast_model.pkl', 'rb'))

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')
df['ID']='test'
df.tail()

,ds,y,ID
4,2022-07-30,219,test
3,2022-07-31,195,test
2,2022-08-01,283,test
1,2022-08-02,300,test
0,2022-08-03,269,test


In [8]:

future = loaded_model.make_future_dataframe(df, periods=8) # periods=m.n_forecasts, n_historic_predictions=False

forecast = loaded_model.predict(future, raw=True, decompose=False)

start = forecast.values.tolist()[-1][0]
forecast_length = len(forecast.values.tolist()[-1][1:])

forecast_output = pd.DataFrame(columns=['ds','visits','timestamp'])
forecast_output['ds'] = pd.date_range(start=start, periods=forecast_length, freq='D')
forecast_output['visits'] = forecast.values.tolist()[-1][1:]
forecast_output['timestamp'] = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output.to_csv('forecasts/daily_visits.csv', index=False)


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.947% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (NP.forecaster.predict) - Raw forecasts are incompatible with plotting utilities
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 97.297% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 97.297% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


OSError: Cannot save file into a non-existent directory: 'forecasts'

In [5]:
future

,ds,y
0,2022-07-06,274
1,2022-07-07,265
2,2022-07-08,287
3,2022-07-09,203
4,2022-07-10,256
5,2022-07-11,291
6,2022-07-12,267
7,2022-07-13,271
8,2022-07-14,277
9,2022-07-15,275
